In [1]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import dash_leaflet as dl
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import mapping
import leafmap as leafmap
from shapely.geometry import shape
from dash_extensions.javascript import assign
from dash import Output, Input
from datetime import datetime, date


Useful Functions!

In [2]:
# read_response checks if the response of the server is valid JSON

def read_response(t):
    try:
        data = t.json() #This will convert the response to a json object
        return data
    except requests.exceptions.JSONDecodeError:
        print("Risposta non valida JSON!")
        print("Contenuto della risposta:", t.text)
        data = None 

# get_measurement_unit returns the measurement unit of a given pollutant

def get_measurement_unit(pollutant):
    t=requests.post(url="http://127.0.0.1:5000/api/units", json={"var_pollutant": pollutant}) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    return data[0]

# get_province_shape returns a GeoDataFrame containing the geometries of the provinces
   
def get_province_shape():
    t=requests.get(url="http://127.0.0.1:5000/api/province_shape")
    data = read_response(t)
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry_province'] = gdf['geometry_province'].apply(wkt.loads)
    gdf.set_geometry('geometry_province', inplace=True)
    return gdf

# list_sensors_and_pollutants gets the list of sensor and relative pollutants for a given station

def list_sensors_and_pollutants(list_stations):
    t=requests.post(url="http://127.0.0.1:5000/api/sensors_and_pollutants", json={"var_id_stazione": list_stations}) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    return data

# get_station_location returns a GeoDataFrame containing the points of the stations

def get_station_location():
    t=requests.get(url="http://127.0.0.1:5000/api/station_location")
    data = read_response(t)
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
    gdf.set_geometry('geometry', inplace=True)
    return gdf

############################################# FUNCTION TO POPULATE THE DROPDOWN #############################################

# pollutant_dropdown populates the dropdown with the pollutatnts that have at least one recorded value

def pollutant_dropdown():
    t=requests.get(url="http://127.0.0.1:5000/api/pollutant")
    list_pollutant = read_response(t)
    list = [{'label': pollutant, 'value': pollutant} for pollutant in list_pollutant]
    return list

# all_pollutant_dropdown populates the dropdown with all the pollutants, even those that have no recorded values

def all_pollutant_dropdown():
    t=requests.get(url="http://127.0.0.1:5000/api/all_pollutant")
    list_pollutant = read_response(t)
    list = [{'label': pollutant, 'value': pollutant} for pollutant in list_pollutant]
    return list

############################################# FUNCTION FOR MIN E MAX DATES #############################################

#Funzione per estrarre mindata e maxdata una volta specificato il pollutant
def get_data_eudv2(pollutant):
    data = {"var_pollutant": pollutant,
            "var_start_date": "1900-01-01 00:00:00",
            "var_end_date"  : "2100-01-01 23:59:59",
            "var_province"  : "",          # oppure lascia stringa vuota
            "var_threshold" : -9999         # soglia impossibile
            }
    t = requests.post(url="http://127.0.0.1:5000/api/EU_DV_2", json=data)
    data = read_response(t)
    df = pd.DataFrame(data)
    print(">>> contenuto ricevuto:", data[:2])   # stampa i primi 2 record
    df['data'] = pd.to_datetime(df['data'], errors='coerce')
    min_date = df["data"].min().date()
    max_date = df["data"].max().date()
    print(t.json()[:2])          # vedi i primi 2 record
    print(df.columns.tolist())   # vedi i nomi colonna effettivi
    return min_date, max_date

def fetch_pollutant_date_range(pollutant):
    try:
        r = requests.post(
            "http://127.0.0.1:5000/api/EU_DV_2/date_range",
            json={"var_pollutant": pollutant},
            timeout=5
        )
        r.raise_for_status()
        data = r.json()

        if not data.get("start_date") or not data.get("end_date"):
            print("Nessuna data trovata per", pollutant)
            return date(2000, 1, 1), date(2025, 12, 31)

        # Usa datetime.fromisoformat per gestire orario
        min_d = datetime.fromisoformat(data["start_date"]).date()
        max_d = datetime.fromisoformat(data["end_date"]).date()
        return min_d, max_d

    except Exception as e:
        print("Errore fetch_pollutant_date_range:", e)
        return date(2000, 1, 1), date(2025, 12, 31)

# Utility Function to Read API Responses

def read_response(response):
    try:
        return response.json()
    except requests.exceptions.JSONDecodeError:
        print("Invalid JSON response!")
        print("Response content:", response.text)
        return None
    
# Get List of Available Provinces

def get_provinces_list():
    response = requests.get("http://127.0.0.1:5000/api/provinces")
    return read_response(response) or []


# Get List of Available Pollutants

def get_pollutants_list():
    response = requests.get("http://127.0.0.1:5000/api/pollutants")
    return read_response(response) or []

# Get startdate and enddata possible

def get_data_dict():
    response = requests.get("http://127.0.0.1:5000/api/data")
    return read_response(response) or []

def update_time_series(n_clicks, pollutant, province, start_date, end_date, threshold):
    if not all([pollutant, province, start_date, end_date, threshold]):
        return {}, " Please select all inputs."
    payload = {
        "var_pollutant": pollutant,
        "var_start_date": f"{start_date} 00:00:00",
        "var_end_date": f"{end_date} 23:59:59",
        "var_province": province,
        "var_threshold": threshold
    }

    try:
        response = requests.post("http://127.0.0.1:5000/api/EU_DV_2", json=payload)
        response.raise_for_status()
        data = response.json()
        if not data:
            return {}, "No exceedance data found."

        df = pd.DataFrame(data)
        df['data'] = pd.to_datetime(df['data'])
        df = df.sort_values('data')

        # Raggruppa per giorno
        df_grouped = df.groupby(df['data'].dt.date)['valore'].max().reset_index()
        df_grouped.columns = ['Data', 'Valore max']

        # Calcolo percentuale
        unique_days = df['data'].dt.date.nunique()
        total_days = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days + 1
        percent = (unique_days / total_days) * 100

        fig = px.bar(df_grouped, x='Data', y='Valore max', title="Days exceeding threshold")
        fig.add_hline(y=threshold, line_dash="dash", line_color="blue", annotation_text=f"Threshold = {threshold}")

        return fig, f" Exceeded on {unique_days} out of {total_days} days ({percent:.1f}%)"

    except Exception as e:
        return {}, f" Error: {str(e)}"
    
def update_slider_limits(pollutant):
    """
    Ogni volta che cambia il pollutant, recupera min/max dal backend
    e aggiorna lo slider.  Imposta il valore corrente al midpoint.
    """
    if not pollutant:
        # se nulla selezionato → valori di default
        default_min, default_max = 0, 80
        marks = {i: str(i) for i in range(default_min, default_max+1, 20)}
        return default_min, default_max, 50, marks

    min_val, max_val = fetch_threshold_range(pollutant)

    # assicura che min < max
    if min_val >= max_val:
        min_val, max_val = 0, 80

    # valore corrente = metà tra min e max
    current_val = round((min_val + max_val) / 2, 1)

    # genera le marks ogni 20 µg/m³ (o altro step a piacere)
    step_mark = max(1, int((max_val - min_val) // 5))
    marks = {i: str(i) for i in range(int(min_val), int(max_val)+1, step_mark)}

    return min_val, max_val, current_val, marks

def update_date_range_limits(pollutant):
    if pollutant:
        return fetch_pollutant_date_range(pollutant)
    return date(2000, 1, 1), date(2025, 12, 31)

def fetch_threshold_range(pollutant):
    """
    Chiama l’endpoint e ritorna (min_val, max_val).
    Se non ottiene dati, ritorna valori di fallback (0, 80).
    """
    try:
        r = requests.post(
            "http://127.0.0.1:5000/api/EU_DV_2/threshold_range",
            json={"var_pollutant": pollutant},
            timeout=5
        )
        r.raise_for_status()
        data = r.json()

        if data.get("min_val") is None or data.get("max_val") is None:
            print(f" Nessun valore trovato per {pollutant}")
            return 0, 80

        return float(data["min_val"]), float(data["max_val"])

    except Exception as e:
        print("Errore fetch_threshold_range:", e)
        return 0, 80



Definition of variables for visualization

In [ ]:

############################################# EU_DV_2 #############################################

app = Dash(__name__)

app.layout = html.Div([
    html.H1('Bugs_project: Air quality analysis'),
    html.P('Description of the dashboard functionalities'),

    html.Div([
        html.H2('EU_DV_2 Threshold definition – time-series'),
        html.P("Select a pollutant, province, date range, and threshold to visualize which days exceeded it."),

        dcc.Dropdown(
            id='all-pollutant-dropdown_EU_DV_2',
            options=get_pollutants_list(),
            placeholder="Select a pollutant",
            style={'margin-top': 10}
        ),

        dcc.Dropdown(
            id='province-dropdown_EU_DV_2',
            options=get_provinces_list(),
            placeholder="Select a province",
            style={'margin-top': 10}
        ),

        dcc.DatePickerRange(
            id='date-range-picker_EU_DV_2',
            min_date_allowed=date(2000, 1, 1),
            max_date_allowed=date(2025, 12, 31),
            start_date_placeholder_text="Start Date",
            end_date_placeholder_text="End Date",
            display_format='YYYY-MM-DD',
            style={'margin-top': 10}
        ),

        html.Div([
            html.Label("Threshold:"),
            dcc.Slider(
                id='threshold-slider_EU_DV_2',
                min=0,
                max=80,
                step=1,
                value=50,
                marks={i: str(i) for i in range(0, 201, 20)},
                tooltip={"placement": "bottom", "always_visible": True},
                included=False,
                updatemode='drag'
            )
        ], style={'margin-top': '15px', 'margin-bottom': '30px'}),


        html.Div([
            html.Button(
                'Visualize threshold exceedance',
                id='button_EU_DV_2',
                n_clicks=0,
                style={'margin-top': 20, 'margin-bottom': 20}
            )
        ], style={'textAlign': 'center'}),

        html.Div(id='exceedance-percent_EU_DV_2', style={'textAlign': 'center'}),
        dcc.Graph(id='histogram-output_EU_DV_2')
    ])
])

############################################# Callback EU_DV_2 #############################################

@app.callback(
    Output('histogram-output_EU_DV_2', 'figure'),
    Output('exceedance-percent_EU_DV_2', 'children'),
    Input('button_EU_DV_2', 'n_clicks'),
    State('all-pollutant-dropdown_EU_DV_2', 'value'),
    State('province-dropdown_EU_DV_2', 'value'),
    State('date-range-picker_EU_DV_2', 'start_date'),
    State('date-range-picker_EU_DV_2', 'end_date'),
    State('threshold-slider_EU_DV_2', 'value')
)
def update_time_series(n_clicks, pollutant, province, start_date, end_date, threshold):
    if not all([pollutant, province, start_date, end_date, threshold]):
        return {}, " Please select all inputs."

    payload = {
        "var_pollutant": pollutant,
        "var_start_date": f"{start_date} 00:00:00",
        "var_end_date": f"{end_date} 23:59:59",
        "var_province": province,
        "var_threshold": threshold
    }

    try:
        response = requests.post("http://127.0.0.1:5000/api/EU_DV_2", json=payload)
        response.raise_for_status()
        data = response.json()
        if not data:
            return {}, "No exceedance data found."

        df = pd.DataFrame(data)
        df['data'] = pd.to_datetime(df['data'])
        df = df.sort_values('data')

        # Raggruppa per giorno
        df_grouped = df.groupby(df['data'].dt.date)['valore'].max().reset_index()
        df_grouped.columns = ['Data', 'Valore max']

        # Calcolo percentuale
        unique_days = df['data'].dt.date.nunique()
        total_days = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days + 1
        percent = (unique_days / total_days) * 100

        fig = px.bar(df_grouped, x='Data', y='Valore max', title="Days exceeding threshold")
        fig.add_hline(y=threshold, line_dash="dash", line_color="blue", annotation_text=f"Threshold = {threshold}")

        return fig, f" Exceeded on {unique_days} out of {total_days} days ({percent:.1f}%)"

    except Exception as e:
        return {}, f" Error: {str(e)}"

@app.callback(
    [Output('date-range-picker_EU_DV_2', 'min_date_allowed'),
     Output('date-range-picker_EU_DV_2', 'max_date_allowed')],
    Input('all-pollutant-dropdown_EU_DV_2', 'value')
)

def fetch_pollutant_date_range(pollutant):
    try:
        r = requests.post(
            "http://127.0.0.1:5000/api/EU_DV_2/date_range",
            json={"var_pollutant": pollutant},
            timeout=5
        )
        r.raise_for_status()
        data = r.json()

        if not data.get("start_date") or not data.get("end_date"):
            print("Nessuna data trovata per", pollutant)
            return date(2000, 1, 1), date(2025, 12, 31)

        # Usa datetime.fromisoformat per gestire orario
        min_d = datetime.fromisoformat(data["start_date"]).date()
        max_d = datetime.fromisoformat(data["end_date"]).date()
        return min_d, max_d

    except Exception as e:
        print("Errore fetch_pollutant_date_range:", e)
        return date(2000, 1, 1), date(2025, 12, 31)


@app.callback(
    Output('threshold-slider_EU_DV_2', 'min'),
    Output('threshold-slider_EU_DV_2', 'max'),
    Output('threshold-slider_EU_DV_2', 'value'),   # imposta valore di default
    Output('threshold-slider_EU_DV_2', 'marks'),   # opzionale: aggiorna etichette
    Input('all-pollutant-dropdown_EU_DV_2', 'value')
)

def update_date_range_limits(pollutant):
    if pollutant:
        return fetch_pollutant_date_range(pollutant)
    return date(2000, 1, 1), date(2025, 12, 31)

def fetch_threshold_range(pollutant):
    """
    Chiama l’endpoint e ritorna (min_val, max_val).
    Se non ottiene dati, ritorna valori di fallback (0, 80).
    """
    try:
        r = requests.post(
            "http://127.0.0.1:5000/api/EU_DV_2/threshold_range",
            json={"var_pollutant": pollutant},
            timeout=5
        )
        r.raise_for_status()
        data = r.json()

        if data.get("min_val") is None or data.get("max_val") is None:
            print(f" Nessun valore trovato per {pollutant}")
            return 0, 80

        return float(data["min_val"]), float(data["max_val"])

    except Exception as e:
        print("Errore fetch_threshold_range:", e)
        return 0, 80


def update_slider_limits(pollutant):
    """
    Ogni volta che cambia il pollutant, recupera min/max dal backend
    e aggiorna lo slider.  Imposta il valore corrente al midpoint.
    """
    if not pollutant:
        # se nulla selezionato → valori di default
        default_min, default_max = 0, 80
        marks = {i: str(i) for i in range(default_min, default_max+1, 20)}
        return default_min, default_max, 50, marks

    min_val, max_val = fetch_threshold_range(pollutant)

    # assicura che min < max
    if min_val >= max_val:
        min_val, max_val = 0, 80

    # valore corrente = metà tra min e max
    current_val = round((min_val + max_val) / 2, 1)

    # genera le marks ogni 20 µg/m³ (o altro step a piacere)
    step_mark = max(1, int((max_val - min_val) // 5))
    marks = {i: str(i) for i in range(int(min_val), int(max_val)+1, step_mark)}

    return min_val, max_val, current_val, marks

if __name__ == '__main__':
    print("Starting Dashboard...")
    app.run(port=8089, debug=True)




Starting Dashboard...


[2025-06-25 08:54:38,968] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\Davide\anaconda3\envs\se4g_project\Lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\Davide\anaconda3\envs\se4g_project\Lib\site-packages\flask\app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "c:\Users\Davide\anaconda3\envs\se4g_project\Lib\site-packages\dash\dash.py", line 1414, in dispatch
    ctx.run(
    ~~~~~~~^
        functools.partial(
        ^^^^^^^^^^^^^^^^^^
    ...<7 lines>...
        )
        ^
    )
    ^
  File "c:\Users\Davide\anaconda3\envs\se4g_project\Lib\site-packages\dash\_callback.py", line 554, in add_context
    flat_output_values = flatten_grouping(output_value, output)
  File "c

Errore fetch_pollutant_date_range: 400 Client Error: BAD REQUEST for url: http://127.0.0.1:5000/api/EU_DV_2/date_range


[2025-06-25 08:54:48,062] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\Davide\anaconda3\envs\se4g_project\Lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\Davide\anaconda3\envs\se4g_project\Lib\site-packages\flask\app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "c:\Users\Davide\anaconda3\envs\se4g_project\Lib\site-packages\dash\dash.py", line 1414, in dispatch
    ctx.run(
    ~~~~~~~^
        functools.partial(
        ^^^^^^^^^^^^^^^^^^
    ...<7 lines>...
        )
        ^
    )
    ^
  File "c:\Users\Davide\anaconda3\envs\se4g_project\Lib\site-packages\dash\_callback.py", line 554, in add_context
    flat_output_values = flatten_grouping(output_value, output)
  File "c